# Quantitative Trading System

This notebook integrates the following components:
1. Order Book Simulation
2. Mean Reversion Strategy
3. Backtesting Framework

## Imports and Setup

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from collections import deque
import sys
import os
%run order_book.py
# Add the current directory to path to ensure imports work
sys.path.append(os.path.abspath('.'))

# Import our custom modules
from order_book import OrderBook
from mean_reversion import MeanReversionStrategy

## 1. Order Book Demonstration

In [15]:
# Create an instance of the order book
book = OrderBook()

# Add some sample orders
print("Adding orders to the book...")
book.add_order(price=100.0, quantity=10, side='buy')
book.add_order(price=101.0, quantity=5, side='buy')
book.add_order(price=99.0, quantity=7, side='buy')
book.add_order(price=102.0, quantity=3, side='sell')
book.add_order(price=103.0, quantity=8, side='sell')
book.add_order(price=101.5, quantity=4, side='sell')

# Print the current order book state
book.output()

# Match orders
print("\nMatching orders...")
book.match_orders()

# Print the order book after matching
print("\nOrder book after matching:")
book.output()

Adding orders to the book...

Current order book:
Bids (Buy orders):
1. Price: 101.0, Quantity: 5
2. Price: 100.0, Quantity: 10
3. Price: 99.0, Quantity: 7

Asks (Sell orders):
1. Price: 101.5, Quantity: 4
2. Price: 102.0, Quantity: 3
3. Price: 103.0, Quantity: 8

Matching orders...

Order book after matching:

Current order book:
Bids (Buy orders):
1. Price: 101.0, Quantity: 5
2. Price: 100.0, Quantity: 10
3. Price: 99.0, Quantity: 7

Asks (Sell orders):
1. Price: 101.5, Quantity: 4
2. Price: 102.0, Quantity: 3
3. Price: 103.0, Quantity: 8


## 2. Mean Reversion Strategy

In [16]:
# Load historical stock data
print("Loading historical stock data...")
stock_data = yf.download('AAPL', start='2020-01-01', end='2022-01-01')
print(f"Loaded {len(stock_data)} days of data")
stock_data.head()

[*********************100%***********************]  1 of 1 completed

Loading historical stock data...
Loaded 505 days of data


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-01-02,72.716072,72.776598,71.466812,71.721019,135480400
2020-01-03,72.009117,72.771745,71.783962,71.941328,146322800
2020-01-06,72.582924,72.621661,70.876090,71.127881,118387200
2020-01-07,72.241508,72.849185,72.021193,72.592555,108872000
2020-01-08,73.403648,73.706279,71.943759,71.943759,132079200


In [20]:
data = stock_data['Close']
data

Ticker,AAPL
Date,
2020-01-02,72.716072
2020-01-03,72.009117
2020-01-06,72.582924
2020-01-07,72.241508
2020-01-08,73.403648
...,...
2021-12-27,177.228790
2021-12-28,176.206680
2021-12-29,176.295181


In [ ]:

# Create and apply the mean reversion strategy
strategy = MeanReversionStrategy(stock_data['Close'])
signals = strategy.generate_signals()
stock_data['Signal'] = signals

# Display the first few rows of the data with signals
print("\nStock data with signals:")
display(stock_data.head())

# Plot the strategy
plt.figure(figsize=(14, 7))
plt.plot(stock_data['Close'], label='Stock Price', alpha=0.7)
plt.plot(strategy.mean, linestyle='-', color='blue', label='Mean', alpha=0.5)
plt.plot(strategy.upper_band, linestyle='dashed', color='red', label='Upper Bound')
plt.plot(strategy.lower_band, linestyle='dashed', color='green', label='Lower Bound')

# Mark buy and sell signals
buy_signals = stock_data[stock_data['Signal'] == 'BUY']
sell_signals = stock_data[stock_data['Signal'] == 'SELL']

plt.scatter(buy_signals.index, buy_signals['Close'], color='green', marker='^', s=100, label='Buy')
plt.scatter(sell_signals.index, sell_signals['Close'], color='red', marker='v', s=100, label='Sell')

plt.title('Mean Reversion Strategy for AAPL')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

ValueError: Length of values (0) does not match length of index (505)

## 3. Backtesting

In [ ]:
# Implement backtesting
def backtest_strategy(stock_data, initial_capital=10000):
    capital = initial_capital
    position = 0
    trades = []
    portfolio_value = [initial_capital]
    
    for i in range(len(stock_data)):
        current_value = capital if position == 0 else position * stock_data['Close'].iloc[i]
        portfolio_value.append(current_value)
        
        if stock_data['Signal'].iloc[i] == 'BUY' and capital > 0:
            position = capital / stock_data['Close'].iloc[i]
            trades.append((stock_data.index[i], 'BUY', stock_data['Close'].iloc[i], position))
            capital = 0
        elif stock_data['Signal'].iloc[i] == 'SELL' and position > 0:
            capital = position * stock_data['Close'].iloc[i]
            trades.append((stock_data.index[i], 'SELL', stock_data['Close'].iloc[i], position))
            position = 0
    
    final_value = capital if capital > 0 else position * stock_data['Close'].iloc[-1]
    return final_value, trades, portfolio_value[1:]

# Run the backtest
final_value, trades, portfolio_values = backtest_strategy(stock_data)

# Print results
print(f"Initial Capital: $10,000.00")
print(f"Final Portfolio Value: ${final_value:.2f}")
print(f"Return: {((final_value / 10000) - 1) * 100:.2f}%")
print(f"Number of trades: {len(trades)}")

# Display trade history
print("\nTrade History:")
for date, action, price, shares in trades[:10]:  # Show first 10 trades
    print(f"{date.date()} - {action}: {shares:.2f} shares at ${price:.2f}")
if len(trades) > 10:
    print(f"... and {len(trades) - 10} more trades")

# Plot portfolio performance
plt.figure(figsize=(14, 7))
plt.plot(stock_data.index, portfolio_values, label='Portfolio Value', color='blue')

# Add buy/sell markers
for date, action, price, shares in trades:
    if action == 'BUY':
        plt.scatter(date, price * shares, color='green', marker='^', s=100)
    else:  # SELL
        plt.scatter(date, price * shares, color='red', marker='v', s=100)

plt.title('Portfolio Performance')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## 4. Strategy Performance Analysis

In [ ]:
# Calculate performance metrics
def calculate_metrics(portfolio_values, stock_data):
    # Convert to numpy arrays for calculations
    portfolio_returns = np.array([(portfolio_values[i] - portfolio_values[i-1]) / portfolio_values[i-1] 
                               for i in range(1, len(portfolio_values))])
    
    # Calculate metrics
    total_return = (portfolio_values[-1] / portfolio_values[0]) - 1
    annual_return = (1 + total_return) ** (252 / len(portfolio_returns)) - 1
    daily_std = np.std(portfolio_returns)
    sharpe_ratio = np.mean(portfolio_returns) / daily_std * np.sqrt(252)  # Annualized Sharpe
    max_drawdown = np.max(np.maximum.accumulate(portfolio_values) - portfolio_values) / np.max(portfolio_values)
    
    # Buy & Hold comparison
    buy_hold_return = (stock_data['Close'].iloc[-1] / stock_data['Close'].iloc[0]) - 1
    
    return {
        'Total Return': total_return,
        'Annual Return': annual_return,
        'Daily Volatility': daily_std,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown,
        'Buy & Hold Return': buy_hold_return
    }

# Calculate and display the metrics
metrics = calculate_metrics(portfolio_values, stock_data)

print("Performance Metrics:")
print(f"Total Return: {metrics['Total Return'] * 100:.2f}%")
print(f"Annual Return: {metrics['Annual Return'] * 100:.2f}%")
print(f"Daily Volatility: {metrics['Daily Volatility'] * 100:.2f}%")
print(f"Sharpe Ratio: {metrics['Sharpe Ratio']:.2f}")
print(f"Maximum Drawdown: {metrics['Max Drawdown'] * 100:.2f}%")
print(f"Buy & Hold Return: {metrics['Buy & Hold Return'] * 100:.2f}%")

# Compare strategy vs. Buy & Hold
plt.figure(figsize=(14, 7))

# Normalize values to start at 100
norm_portfolio = [v * 100 / portfolio_values[0] for v in portfolio_values]
norm_stock = stock_data['Close'] * 100 / stock_data['Close'].iloc[0]

plt.plot(stock_data.index, norm_portfolio, label='Strategy', linewidth=2)
plt.plot(stock_data.index, norm_stock, label='Buy & Hold', linewidth=2, alpha=0.7)

plt.title('Strategy vs. Buy & Hold Performance (Normalized)')
plt.xlabel('Date')
plt.ylabel('Value (Starting at 100)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()